## Human pose estimation(HPE)
- 크게 2D와 3D로 나뉜다
- 2D HPE는 2D 이미지에서 (x, y) 2차원 좌표들을 찾아내고,
- 3D HPE는 2D 이미지에서 (x, y, z) 3차원 좌표들을 찾아낸다.
<br/><br/>

```
목적
    - 사람의 전체 자세를 이해하고 각 부분(관절)의 위치를 찾는 것
    - 즉, 사람의 머리, 어깨, 팔꿈치, 손목, 무릎, 발목 등 다양한 관절들을 찾아내어 전체적인 자세를 파악
```

## 22-3. Pose는 face landmark 랑 비슷하다.
- 이미 2D pose estimation은 face landmark랑 비슷하다.
- 입력과 출력이 개수만 다를 뿐 비슷하다.
<br/><br/>
- 하지만 난이도에서 차이가 난다.
- face landmark는 물리적으로 거의 고정되어 있는 (입이 얼굴보다 클 수 없다.) 반면,
- human pose는 팔, 다리가 상대적으로 넓은 범위와 자유도를 갖는다는 것을 고려해야 한다.

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/02_8LQHgwE.png)
- 자유도가 높다는 것은 데이터 분포를 특정하기 어렵다는 볼 수 있다.
- 데이터 분포를 학습하기 어렵다면 당연히 학습에 더 많은 데이터가 필요하고 더 복잡한 모델을 사용해야 한다는 것을 의미한다.
<br/><br/>
- 따라서 상당히 많은 사전 작업이 요구되고
- 사용하려는 애플리케이션에 따라 접근 방법도 달라진다.
<br/><br/>
- 가장 초기에 만나는 접근법은 두 가지로 나눠진다.

### 우리에게 맞는 방법은 뭘까?
![image](https://d3s0tskafalll9.cloudfront.net/media/images/03_6C5RZR1.max-800x600.png)
1. Top-down 방법
    - 모든 사람의 정확한 keypoint를 찾기 위해 obejct detection을 사용한다.
    - crop 한 이미지 내에서 keypoint를 찾아내는 방법으로 표현한다.
    - detector가 선행되어야 하고 모든 사람마다 알고리즘을 적용해야 하기 때문에 사람이 많이 등장할 때는 느리다는 단점이 있다.

2. Bottom-up 방법
    - detector가 없고 keypoint를 먼저 검출한다. (예를 들어 손목에 해당하는 모든 점들을 검출한다.)
    - 한 사람에 해당하는 keypoint를 clustering 한다.
    - detecotr가 없기 때문에 다수의 사람이 영상에 등장하더라도 속도 저하가 크지 않다.
    - 반면 top down 방식에 비해 keypoint 검출 범위가 넓어 성능이 떨어진다는 단점이 있다.

```
1. Top-down
    1. 모든 사람에 대해 object detection을 한다.
    2. crop하여 원하는 부분을 잘라준다.
    3. 해당 하는 이미지에서 keypoint를 찾아낸다.

2. Bottom-up
    1. keypoint를 먼저 검출한다.
    2. 검출된 keypoint들을 개별적으로 감지되며, 같은 사람에 해당하는 keypoint들을 클러스터링한다.
    - 예를 들어 여러 손목 keypoint들을 하나의 그룹으로 묶어 특정 사람에 해당하는 keypoint들을 식별한다.
    3. 클러스터링된 keypoint들을 기반으로 각 사람의 전체적인 구조를 구성한다.
```

## 22-4. human keypoint detection (1)
### 자유도가 높은 사람의 동작
- human pose estimation은 keypoint의 localization 문제를 푼다는 점에서 비슷하다.
- 하지만 손목, 팔꿈치 등의 joint keypoint 정보는 얼굴의 keypoint 보다 훨씬 다양한 위치의 변화를 보인다.

![image](https://github.com/Team-Neighborhood/Kalman-Filter-Image/raw/master/result/KF_result.gif)
- 위 이미지에서 볼 수 있듯이,
- 손이 얼굴을 가리는 행위, 모든 keypoint가 영상에 담기지 않는 등
- invisible, occlusions, clothing, lighing change가 face landmark에 비해 더 어려운 환경을 만들어 낸다.
<br/><br/>
- 딥러닝 기반 방법이 적용되기 전에는 다양한 사전 지식이 사용되었다.
- 가장 기본이 되는 아이디어는
    - "인체는 변형 가능 부분으로 나누어져 있고 각 부분끼리 연결성을 가지고 있다."

![image](https://d3s0tskafalll9.cloudfront.net/media/images/05_9oDmNOY.max-800x600.png)
- 그림에서 보이는 것처럼 손은 팔, 팔은 몸과 연결되어 있다.
- 손이 다리 옆에 있을 확률이 팔 옆에 있을 확률보다 훨씬 작을 것이다.
- 이런 제약 조건을 그림에 보이는 스프링으로 표현했다.
<br/><br/>
- 3D 환경에서 생각하면 정말 좋은 방법이다.
- 하지만 우리가 다루는 데이터는 2D 이지미 데이터이기 때문에 촬영 각도에 따라 충분히 팔이 다리 옆에서 관찰될 수 있다.
<br/><br/>
- 이 문제를 해결하기 위해 Deformable part mopdels 방법에서는 각 부분(part) 들의 complex joint relationship의 mixture model로 keypoint를 표현하는 방법을 이용했지만 성능은 사람들의 기대에 미치지 못했다.

### DeepPose
- 딥러닝 이전의 전통적 pose estimation 모델은 분명한 한계가 있다.
- graphical tree model은 같은 이미지에 두 번 연산을 하는 등 연산 효율이 떨어지는 점과  그에 비해서도 부족한 성능이 문제점으로 인식되어 왔다.
<br/><br/>
- AlexNet 이후, 다양한 분야에 CNN이 적용되면서 pose estimation 분야에도 CNN을 이용한 방법이 나타나기 시작했다.
- Toshev and Szegedy는 처음으로 딥러닝 기반 keypoint localization 모델을 제안했다.

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/06_ihhZoAI.png)
- 기존 기술로는 풀기 어렵웠던 동작의 다양성,
- invisible joint의 문제를 언급하며 딥러닝 기반 추론 방법이 해결책이 될 수 있다는 것을 증명했다.

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/07_GOoFtrb.png)
- 초기의 pose estimation 모델은 x, y 좌표를 직접적으로 예측하는 position regression 문제로 인식했다.
- human detection을 통한 crop된 사람 이미지를 이용해서 딥러닝 모델에 입력하고 (x, y) 좌표를 출력하도록 만든다.
<br/><br/>
- DeepPose는 매우 혁신적인 시도였던 것에 비해 사실 성능이 압도적으로 높았던 것은 아니다.

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/08_iBnIDWo.png)
- DeepPose가 전반적으로 높은 성능을 나타내고 있기 하지만
- 기존 Tree based model 인 Wang et al.의 방법에 비해 비약적으로 성능을 상승시켰다고 말하기는 어렵다.
- DeepPose의 기여는 SOTA에 가까운 성능을 내면서도 딥러닝을 적용한 첫 번째 사례라고 할 수 있다.

```
pose estimation(자세 추정) 분야
    - 자세를 이해하고 추정하는 분야
    - human pose estimation - 사람의 관절 위치를 감지하고 추정하는 작업
    - object pose estimation - 물체의 자세를 추정하는 작업

keypoint localization 모델
    - 이미지나 비디오에서 특정 객체나 인체 부분의 keypoint(주요 지점)를 찾는 모델

invisible joint(투명한 과절)
    - 환경이나 다른 객체에 의해 관절이 가려진 경우 발생하는 현상

position regression인 DeepPose는 L2 loss 함수를 사용했다.
    - L2 loss는 예측 값과 실제 값 사이의 유클리드 거리를 측정한다.
    - 예측 값과 실제 값 간의 차이를 제곱한 후 그 합을 구하는 것
```

### Efficient Object Localization Using Convolutional Network
- DeepPose는 딥러닝을 사용했는데 왜 성능이 비약적으로 상승하지 않았을까?
- Tompson이 제안한 Efficient object localization 방법을 보자.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/09_NSfxHyY.max-800x600.png)
- 제안한 모델이 DeepPose에 비해 깊어졌지만,
- 가장 중요한 건 keypoint의 위치를 직접 예측하기보다 keypoint가 존재할 확률 분포를 학습하게 하자는 점이다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/10_SBHBawM.max-800x600.png)
- human pose (keypoint)도 사람이 labeling을 할 수 밖에 없는데 사람이 항상 같은 위치의 점을 찍을 수 있을까?
- 아래 동영상에서
- 귀는 귀에
- 눈은 눈에
- 어깨는 어깨에
- keypoint 자체는 잘 찍혀 있지만 어색하다.

![image](https://github.com/Team-Neighborhood/Kalman-Filter-Image/raw/master/result/KF_result.gif)
- Kalman filter라고 적힌 동영상에 비해
- Orig measured는 점이 굉장히 떨리고 있다.
- 항상 같은 위치라고 생각하면서 keypoint를 선택하지만 사실 매 사진마다 수 픽셀씩 차이가 생기고 있다.
- 눈을 찍고 싶다면 눈을 중심으로 "어떤 분포"의 에러가 더해져서 저장되는 것이다.
<br/><br/>
- 자연상태에서 일어나는 확률 분포는 가우시안 분포일 가능성이 크다.

```
가우시안 분포
    - 정규 분포
    - 평균 주위의 데이터가 모여있는 경향이 있음
    - 종 모양의 대칭적인 분
```

- Tomposn은 이런 점에 착안하여 label을 (x, y) 좌표에서 (x, y)를 중심으로 하는 heatmap으로 변환했다.
- 딥러닝 모델은 이 heatmap을 학습하게 되는 것이다.
- "keypoint가 존재할 확률"을 학습하게 된 이후로 성능이 비약적으로 향상되는 모습을 볼 수 있다.

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/11_bVXlDCF.png)
- Toshev가 제안한 DeepPose에 비해 무려 2배가 넘는 수치를 볼 수 있다.
- 머리의 경우 0.9 가 넘는 높은 성능을 가지게 됐다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/12_VPfjvOz.max-800x600.png)
- MPII 데이터는 2014년에 나온 데이터이다.
- 기존 FLIC 데이터가 머리, 어깨, 팔꿈치, 손목 수준의 적은 개수의 keypoint를 가지고 있었지만
- MPII는 몸의 각 관절 부위 16개의 keypoint를 갖는다.
<br/><br/>
- Tompson이 제안한 방법은 heatmap 학습뿐만 아니다.
- 모델에서도 개선을 이뤘다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/GC-9-L-Casecaded.max-800x600.png)


## 22-5. human keypoint detection (2)
### Convolutional Pose Machines
- CVPR 2016에서 발표된 CPM은 completely differentiable(완전 미분 가능한) 한 multi-stage 구조를 제안했다.
- multi stage 방법들은 DeepPose에서부터 지속적으로 사용되어 왔다.
- 하지만 crop 연산 등 비연속적인 미분 불가능한 stage 단위로 나눠져 있었기 때문에 학습 과정을 여러 번 반복하는 비효율적인 방법을 사용해왔따.
<br/><br/>
- CPM은 end-to-end로 학습할 수 있는 모델을 제안했다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/13_wp3QD5J.max-800x600.png)
- Stage 1은 image feature를 계산하는 역할
- Stage 2는 keypoint를 예측하는 역할
- g1과 g2 모두 heatmap을 출력하게 만들어서 재사용이 가능한 부분은 weight sharing 할 수 있도록 세부 모델을 설계했다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/14.max-800x600.png)
- Stage >= 2에서 볼 수 있듯이 stage 2이상부터는 반복적으로 사용할 수 있다.
- 보통은 3개의 stage를 사용한다고 한다.
1. Stage 1: Image Feature 계산
    - 초기 이미지로부터 중요한 특징을 추출
    - 이는 보통 CNN과 같은 기술을 사용하여 이미지의 기본적인 구조를 파악
2. Stage 2: Keypoint 예측 및 Refinement(정제)
    - Stage 2부터는 반복적으로 사용되는 단계로,
    - 초기 특징에서부터 시작하여 더 정확한 keypoint를 예측하고 이를 반복적으로 계산한다.
    - 예측된 keypoint를 heatmap에 저장한다.
    - heatmap은 다시 입력으로 사용되어 Stage 3에서는 더 정확한 keypoint를 예측하고, 이 과정을 Stage 3이후로 반복한다.
3. Refinement 효과
    - 각 Stage를 거칠수록, 초기에 추정된 keypoint는 미세하게 개선되어 더 정확하 예측이 이루어진다.
    - 입력이 heatmap이 되기 때문에, 각 Stage에서는 더 정교한 이미지 특징이 고려되어 keypoint의 정확성이 향상된다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/15.max-800x600.png)
- CPM이 아주 좋은 방법이라고 말하기 어렵다.
- Multi-stage 방법을 사용하기 때문에 end-to-end로 학습이 가능하더라도 그대로 학습하는 경우는 높은 성능을 달성하기 어렵다.
- 따라서 stage 단위로 pretraining을 한 후 다시 하나의 모델로 합쳐서 학습을 한다.
- 이런 문제점들은 후에 제안되는 모델들이 적극적으로 개선하고 있다.
<br/><br/>
- CPM을 다루는 이유는 성능 때문이다.
- receptive field를 넓게 만드는 multi stage refinement 방법이 성능 향상에 크게 기여한 것 같다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/16.max-800x600.png)
- 주황색 실선이 Tompson 알고리즘이다.
- CPM에서 제안한 검정색, 회색 실선이 detection rate에서 유의미한 차이를 보이고 있는 것을 볼 수 있다.

```
heatmap
특징들이 존재하는 위치
영역안에 있는 픽셀들은 같은 정도의 중요도를 갖는다.
중요한 포인트로부터 얼마나 떨어져있냐에 따라

=> 그걸 어떻게 확인할것이냐 가우시안 분포
    - 중심으로부터 얼마만큼 떨어져있느냐
```

### Stacked Hourglass Network
- ECCV16에서는 DeepPose 이후 랜드마크라고 불릴만한 논문이 제안되었다.<br/>

[논문 : Stacked Hourglass Networks for Human Pose Estimation](https://arxiv.org/pdf/1603.06937.pdf)
<br/><br/>
#### Hourglass
- Stacked Hourglass Network의 기본 구조는 모래시계 같은 모양으로 만들어져 있다.
- Conv layer와 pooling으로 이미지(또는 feature)를 인코딩 하고
- upsampling layer를 통해 feature map의 크기를 키우는 방향으로 decoding한다.
- feature map 크기가 작아졌다 커지는 구조여서 hourglass라고 표현한다.
```
UNet하고 차이가 뭐지?
```

![image](https://d3s0tskafalll9.cloudfront.net/media/images/17.max-800x600.png)
- 기존 방법들과의 가장 큰 차이점은
    1. feature map upsampling
    2. residual connection
- Hourglass은 이미지 처리 과정에서 전역(global) feature과 로컬(local) feature를 고려하여 세밀한 예측을 수행
1. Pooling으로 Global feature 찾기
    - 모델은 이미지를 작게 만들기 위해 pooling과 같은 연산을 사용하여 이미지의 전역 특징을 추출한다.
    - Pooling은 이미지를 격자로 나누고 각 격자에서 특정한 값을 추출하는 연산을 말한다.
    - 이를 통해 전체 이미지에 대한 중요한 특징을 간결하게 표현할 수 있다.
2. Upsampling으로 Local feature 고려
    - 작게 만들어진 이미지에는 global feature이 들어있는데, 이제 이를 다시 크게 만들어 local feature까지 고려해야 한다.
    - Upsampling은 이미지의 크기를 키우는 연산으로, 작게 만들어진 이미지를 다시 원래 크기로 확장한다.
    - 이 단계에서는 local feature이 강조되어 이미지의 세부 정보를 놓치지 않고 복우너한다.
- 이렇게 추출된 global feature과 복원된 local feature을 함께 사용하여 최종 예측을 수행한다.
- 이 아이디어가 hourglass의 핵심적인 독창성이라고 할 수 있다.
<br/><br/>
- Hourglass는 이 간단한 구조를 여러 층으로 쌓아올려서 (stacked) human pose estimation의 성능을 향상시켰다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/18.max-800x600.png)
- MPII에서 처음으로 PCKh@0.5 기준 90%를 넘어서는 성과를 보이게 된다.
- 특유의 간단한 구조와 높은 성능으로 현재까지도 많이 사용되고 있는 구조이다.
---
### SimpleBaseline
- (x, y)를 직접 regression 하는 방법이 heatmap 기반으로 바뀌고 모델의 구조가 바뀌어 가면서 encoder-decoder가 쌓아져 가는 형태가 완성되었다.
- "기술 자체가 많이 발전했는데 현재의 간단한 모델은 얼마나 성능이 좋을까?"
- 이런 생각으로 SimpleBaseline의 저자는 아주 간단한 encoder-decoder 구조를 설계한다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/19.max-800x600.png)
- 그리고 이 구조로 무려 '73.7%'의 AP를 COCO에서 달성한다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/20.max-800x600.png)
- 직전 방법인 hourglass와 직접 비교해 보면, 아래와 같은 결과가 나온다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/21.max-800x600.png)
- ResNet50만 사용한 간단한 구조가 hourglass와 같은 SOTA(state-of-the-art)를 이겼다는 것에 큰 놀라움을 선물한 논문이라고 생각한다.
---
### Deep High-Resolution Network(HRNet)
- HRNet은 개발된 이후 현재까지도 SOTA에 가까운 성능을 보일 정도로 성능이 좋은 알고리즘이다.
- Stacked uourglass, Casecaded pyramid network등은 multi-stage 구조로 이루어져 있어서 학습 & 추론 속도가 느리다는 큰 단점이 있다. (대신 하이퍼파라미터를 최적화할 경우 1-stage 방법보다 성능이 좋다.)
- 반면 Simplebaseline과 HRNet은 간단함을 추구하는 만큼 1-stage를 고수한다.
- 덕분에 구조도 간결해지고 사용하기도 쉽다.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/22.max-800x600.png)
- COCO 데이터셋에서 한때 SOTA의 성능을 자랑했다.
<br/><br/>
- 1-stage에서 어떻게 모델을 변화 시켰을까?
- 기존 알고리즘들을 먼저 살펴보자.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/23.max-800x600.png)
- (a) : Hourglass
- (b) : CPN(cascaded pyramid networks)
- (c) : SimpleBaseline - transposed conv
- (d) : SimpleBaseline - dilated conv
<br/><br/>
- 공통점
    - high resolution -> low resolution인 encoder와
    - low -> high인 decoder 구조로 이루어져 있다.
- 차이점
    - Hourglass는 encoder와 decoder의 비율이 거의 비슷함(대칭적임).
    - 반면 Simplebaseline은 enocder가 무겁고(resnet50 등 backbone 사용) decoder는 가벼운 모델을 사용함.
    - (a), (b)는 skip connection이 있지만 (c)는 skip connection이 없다.
<br/><br/>
- 그렇다면 기존 모델들은 skip connection을 왜 사용했고, Simplebaseline도 사용해야 하지 않을까?
    - pooling(strided conv) 할 때 소실되는 정보를 high level layer에서 사용해서 detail한 정보를 학습하기 위해 사용한다.
    - 당연히 사용할 때 성능이 더 좋을 것 같다.
<br/><br/>
- HRNet 저자도 앞의 질문들에 대해 고민을 했다.
- high -> low -> high의 구조에서 high resolution 정보(representation)을 유지할 수 있는 모델을 어떻게 만들 수 있을까?

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/24.png)
- 고민의 결과, down sample layer를 만들고 작아진 layer feature 정보를 다시 up sampleing 해서 원본 해상도 크기에 적용하는 모델을 제안했다.
- 다소 복잡해 보이지만 1-stage로 동작하기 때문에 전체 flow를 보면 엄청 간단하다.
- CPM이나 Hourglass는 중간 단계에서는 heatmap supervision이 학습과정에 꼭 필요했는데 HRNet은 필요가 없다.
- HRNet 또한 이전 알고리즘들과 마찬가지로 heatmap을 regression하는 방식으로 학습하고 MSE loss를 이용한다.
- 결과를 살펴보자.

![image](https://d3s0tskafalll9.cloudfront.net/media/images/25.max-800x600.png)


## 22-6. 코드로 이해하는 pose estimation
- 이번 노드에서 살펴본 6개의 모델
1. DeepPose
2. Efficient Object Localization
3. CPM(Convolutional Pose Machines)
4. Stacked Hourglass Network
5. SimpleBaseline
6. Deep High-Resolution Network(HRNet)
- 이 6개 모델 중 가장 간단한 모델인 SimpleBaseline을 모델 부부만 정확하게 이해해보자.
---
### SimpleBaseline 구조

![image](https://d3s0tskafalll9.cloudfront.net/media/original_images/26.png)

그림을 통해
- encoder : conv layers
- decoder : deconv module + upsampling
로 이루어져 있다는 것을 알 수 있다.
- 하지만 conv layer가 정확히 어떻게 이루어져 있는지, deconv module은 구성이 어떻게 되어 있는지, deconv module 이 그림처럼 3개일지는 논문을 정확하게 읽어봐야 알 수 있다.

[논문 : Simple Baselines for Human Pose Estimation and Tracking](https://arxiv.org/pdf/1804.06208.pdf)

---
### PyTorch code 읽어보기
- 사실 논문도 모든 디테일을 설명해 주지 않는다.
- 하지만 인공지능 분야의 최대 장점은 저자의 공식 코드가 제공된다는 점이라고 할 수 있다.
- 18년 이후 PyTorch의 급격한 성장으로 절반 이상의 오픈소스가 파이토치로 공개되고 있다.

[https://github.com/Microsoft/human-pose-estimation.pytorch](https://github.com/Microsoft/human-pose-estimation.pytorch)

![image](https://d3s0tskafalll9.cloudfront.net/media/images/27.max-800x600.png)
- 아니나 다를까, 오늘 검토해 보아야 할 공식 코드 repo도 파이토치 기반으로 작성되어 있다.
<br/><br/>
- 파이토치 코드는 케라스 보다 직관적으로 표현되어 있기 때문에 빠르게 파악할 수 있는 장점이 있다.
- 이제 파악한 지식을 이용해서 tf-simplebaseline 모델을 만들어보자.
---


SimpleBaseline - tf2

In [ ]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

94765736/94765736 [==============================] - 1s 0us/step


- deconv module을 만들어보자

In [ ]:
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

- deconv module에서 중복을 제거할 수 없을까?

In [ ]:
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [ ]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

- 이제 각각의 요소를 합쳐 모델을 완성하자

In [ ]:
inputs = keras.Input(shape=(192, 256, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 192, 256, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048   23587712  
                             )                                   
                                                                 
 sequential (Sequential)     (None, 48, 64, 256)       10489600  
                                                                 
 conv2d (Conv2D)             (None, 48, 64, 17)        4369      
                                                                 
Total params: 34081681 (130.01 MB)
Trainable params: 34027025 (129.80 MB)
Non-trainable params: 54656 (213.50 KB)
_________________________________________________________________


- 가상의 이미지를 넣어서 출력이 잘 나오는지 확인하는 코드

In [ ]:
np_input = np.zeros((1,192,256,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=tf.float32)
print('input shape')
print (tf_input.shape)
print('\n')

tf_output = model(tf_input)
print('output shape')
print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

input shape
(1, 192, 256, 3)


output shape
(1, 48, 64, 17)
tf.Tensor(
[[[ 7.07308203e-03 -4.56682872e-03 -5.07805683e-03 -2.98508769e-03
    3.04553454e-04 -7.30087049e-04 -4.59499133e-04 -2.08799099e-03
   -2.51403823e-03 -3.75266885e-03]
  [ 1.02493530e-02  4.41509206e-03 -9.74597502e-03 -6.13354240e-03
    4.35400708e-03  8.69378448e-03 -5.19461511e-03  1.75020229e-02
    1.42028164e-02 -1.68239493e-02]
  [ 1.40832141e-02 -2.43788562e-03 -2.86129466e-03 -7.39622256e-03
    9.04688425e-03 -3.30078020e-03 -1.06298295e-03  4.80880914e-03
    3.50055983e-03 -9.59262252e-03]
  [ 1.83013398e-02 -3.80328391e-04 -9.06329975e-03 -1.68402363e-02
    1.67754292e-02 -1.36420345e-02 -7.66479789e-05  1.82240158e-02
    1.01582939e-02 -2.33027507e-02]
  [ 9.70346853e-03  6.26332173e-03 -1.06528616e-02 -7.21118005e-04
    4.71827434e-03 -5.41052083e-03 -4.59451927e-03  4.83508920e-03
    7.64647545e-03 -2.08331756e-02]
  [ 4.03160322e-03  2.10822443e-03 -8.71996768e-03 -2.00317483e-02
    2.034881